# Prueba técnica ecosistemas

El objetivo principal de este proyecto es implementar una herramienta automatizada que permita a la empresa BATSEJ OPEN FINANCE S.A. calcular mensualmente las comisiones de las cuentas bancarias de sus empresas contratantes de manera rápida, precisa y confiable. Esta automatización no solo optimiza el tiempo invertido en el proceso, sino que también aumenta la fiabilidad de los cálculos y mejora el rendimiento operativo.

Al reducir el esfuerzo manual y minimizar los errores humanos, la automatización garantiza una mayor eficiencia operativa, lo cual se traduce en un claro aumento del retorno sobre la inversión (ROI). Esto no solo mejora la rentabilidad del negocio, sino que también eleva el nivel de satisfacción y fidelización de los clientes, al brindarles un servicio más ágil y confiable.

Además, al implementar esta solución novedosa, BATSEJ OPEN FINANCE S.A. se posiciona de manera más competitiva en el mercado, abriendo la puerta a nuevos proyectos y desafíos. La capacidad de responder con rapidez y precisión a las necesidades de los clientes fortalece la relación comercial, mejora la reputación de la empresa y genera oportunidades para expandir su portafolio de servicios.

### Implementación de librerias 

Para comenzar importarmos las librerías necesarias. Primero, utilizamos sqlite3, ya que nuestra base de datos está en este formato. Esta librería nos permite conectarnos directamente a la base de datos, facilitando la búsqueda, extracción y análisis de las tablas necesarias para el cálculo de comisiones.

A continuación, empleamos pandas, una librería clave de Python para la manipulación de datos. Con pandas, podemos construir y gestionar de manera eficiente diferentes data frames, lo que nos permitirá analizar grandes volúmenes de información de forma estructurada.

Para manejar los archivos en formato Excel, hacemos uso de openpyxl, que facilita tanto la creación como la edición de estos archivos. Esto nos permitirá exportar los resultados del análisis de comisiones en un formato fácil de compartir y presentar.

Utilizamos las librerías smtplib y email.message para automatizar el envío de correos electrónicos. Estas herramientas nos permiten construir un script que envía automáticamente los resultados obtenidos, generando un flujo continuo entre el análisis y la distribución de la información. Así, aseguramos que las empresas eciban la información sobre sus comisiones de manera oportuna y precisa.

Finalmente, las librerias os y dotenv nos permitirán hacer un manejo más responsable de los correos y contraseñas a la hora de enviar el correo electrónico requerido, evitando fechas escritas de forma manual dentro del código, solo será necesario invocar el archivo.env de la carpeta de trabajo donde se alojará dicha información.



In [1]:
# Importamos las librerias requeridas para la implementación del ejercicio

import sqlite3
import pandas as pd
import openpyxl
import smtplib
from email.message import EmailMessage
from dotenv import load_dotenv
import os

In [2]:
path_base = os.getcwd()

# Insumos
path_resultados = os.path.join(path_base, 'Resultados')
path_env = os.path.join(path_base, '.env')

Ahora, implementamos la clase Calculo_comisiones, la cual se diseñó con el propósito de definir las funciones clave que automatizan el cálculo y el cobro de comisiones, cumpliendo con los requerimientos establecidos. A continuación, se explican las características principales de cada función:

- Función ***obtener_datos***: Esta función establece la conexión a la base de datos SQlite permitiendo extraer la información requerida desde las tablas "apicall" y "commerce". El siguiente paso es unificar ambas tablas haciendo uso del lenguaje SQL, en este caso usamos la instrucción left join tomando la tabla apicall como la principal (tabla izquierda) para unir en cada registro los campos commerce_nit, commerce_name, commerce_status y commerce_email de la tabla commerce, se usa como llave de cruce el campo commerce_id. Además,  Además, se filtran los datos según el mes de interés mediante la instrucción WHERE, garantizando que solo se extraiga la información correspondiente al período de cálculo.

- Función ***cobro_peticiones***: Esta función automatiza el cálculo de las comisiones para cada empresa, para ello se requiere ingresar como hiperparametros los campos referentes al número total de peticiones, tanto exitosas como fallidas, así como el IVA y precios referetes a la información suministrada. Cabe resaltar que las condiciones de contrato dadas por las empresas son las que se generan en esta función, se automatizan los campos de tal forma que si se requiere hacer cambios futuros se ingresen dichos cambios a la función sin modificar la estructura del programa. Este diseño flexible asegura que cualquier cambio en los parámetros pueda ser fácilmente integrado al programa, cumpliendo así el objetivo de automatización establecido desde el principio.

- Función ***calcular_comision***:Aquí se genera la tabla final llamada resultado_tab, que consolida para cada empresa el número total de peticiones (exitosas y fallidas), junto con su NIT, nombre y correo electrónico. Esto se logra mediante un cruce (merge) entre la tabla de resultados y la tabla commerce, nuevamente utilizando el campo commerce_id como llave. Con estos datos, se llama a la función cobro_peticiones definida anteriormente, para calcular automáticamente el valor_total, valor_comisión y valor_iva, que constituyen los montos finales a cobrar. Esta tabla final es fundamental para enviar la información consolidada de los clientes. Notemos que como condición se debe cumplir que en el campo commerce_status este en estado "Active" para realizar los cálculos correspondientes. La automatización nos permite realizar los cálculos para las empresas en estado "Inactive" cuando estas cambien su estado, mientras tanto sus valores por defecto serán cero.

- Función ***enviar_correo***: Esta función automatiza el envío de las facturas generadas a las empresas contratantes. Cada factura contiene un contenido detallado de los montos calculados. Para ejecutar esta función, se requieren como parámetros el archivo Excel con los detalles de la comisión, el destinatario y remitente, junto con las credenciales de correo (cuenta y contraseña). En este ejercicio, se spuede encontrar un archivo .env donde se encuentran hospedadas tanto los campos para destinatario, remitente y su contraseña, los cuales pueden ser cambiados por el usuario de acuerdo con la dirección de correo que se quiera  enviar así como desde donde se vaya a realizar. Se usa esta configuración para no exponer información confidencial, a su vez esto nos ayuda aún más a automatizar este proceso.

In [41]:
class Calculo_comisiones:
    
    def __init__(self, conn):
        self.conn = conn
        self.cursor = self.conn.cursor()

    def obtener_datos(self, mes):
        query = f"""
        CREATE TABLE IF NOT EXISTS resultado AS
        SELECT A.*, C.* 
        FROM apicall AS A
        LEFT JOIN commerce AS C
        ON A.commerce_id = C.commerce_id
        WHERE strftime('%Y-%m', A.date_api_call) = '{mes}' 
        """
        self.cursor.execute(query)
        self.conn.commit()

    def cobro_peticiones(self, empresa, peticiones_exitosas, peticiones_fallidas, peticiones_totales, precios, limites,descuentos, porcentajes, iva):
        total = 0
        if empresa == 'Innovexa Solutions':
            total = peticiones_exitosas*precios['Innovexa Solutions'] + peticiones_exitosas*precios['Innovexa Solutions']*iva
        elif empresa == 'NexaTech Industries':
            if peticiones_totales <= limites['NexaTech Industries'][0]:
                total = peticiones_exitosas*precios['NexaTech Industries'][0] + peticiones_exitosas*precios['NexaTech Industries'][0]*iva
            elif peticiones_totales <= limites['NexaTech Industries'][1]:
                total = peticiones_exitosas*precios['NexaTech Industries'][1] + peticiones_exitosas*precios['NexaTech Industries'][1]*iva
            else:
                total = peticiones_exitosas*precios['NexaTech Industries'][2] + peticiones_exitosas*precios['NexaTech Industries'][2]*iva
        elif empresa == 'QuantumLeap Inc.':
            total = peticiones_exitosas*precios['QuantumLeap Inc.'] + peticiones_exitosas*precios['QuantumLeap Inc.']*iva
        elif empresa == 'Zenith Corp.':
            if peticiones_totales >= 0 and peticiones_totales <= limites['Zenith Corp.']:
                total = peticiones_exitosas*precios['Zenith Corp.'][0] + peticiones_exitosas*precios['Zenith Corp.'][0]*iva
            elif peticiones_totales > limites['Zenith Corp.']:
                total = peticiones_exitosas*precios['Zenith Corp.'][1] + peticiones_exitosas*precios['Zenith Corp.'][1]*iva
            if peticiones_fallidas >= descuentos['Zenith Corp.']:
                total = total - total*porcentajes['Zenith Corp.']
        elif empresa == 'FusionWave Enterprises':
            total = peticiones_exitosas*precios['FusionWave Enterprises'] + peticiones_exitosas*precios['FusionWave Enterprises']*iva
            if descuentos['FusionWave Enterprises'][0] <= peticiones_fallidas <= descuentos['FusionWave Enterprises'][1]:
                total = total - total*porcentajes['FusionWave Enterprises'][0]  
            elif peticiones_fallidas > descuentos['FusionWave Enterprises'][1]:
                total = total - total*porcentajes['FusionWave Enterprises'][1]
        return total

    def calcular_comision(self, mes, precios, iva):
        self.obtener_datos(mes)
        
        resultado = pd.read_sql("SELECT * FROM resultado", self.conn)
        commerce = pd.read_sql("SELECT * FROM commerce", self.conn)
        
        resultado_tab = pd.crosstab(resultado['commerce_id'], resultado['ask_status'])
        resultado_tab['Total'] = resultado_tab['Successful'] + resultado_tab['Unsuccessful']

        resultado_tab = resultado_tab.merge(commerce[['commerce_id', 'commerce_name', 'commerce_nit', 'commerce_email', 'commerce_status']], 
                                            left_on='commerce_id', right_on='commerce_id', how='left')
    
        resultado_tab['Cobro Total'] = resultado_tab.apply(
            lambda row: self.cobro_peticiones(row['commerce_name'], row['Successful'], row['Unsuccessful'], row['Total'], precios, limites, descuentos, porcentajes, iva) 
            if row['commerce_status'] == 'Active' else 0, axis=1)
    
    
        resultado_tab['Valor_comision'] = resultado_tab.apply(lambda row: row['Cobro Total'] / (1 + iva) if row['commerce_status'] == 'Active' else 0, axis=1)
        resultado_tab['Valor_iva'] = resultado_tab.apply(lambda row: row['Valor_comision'] * iva if row['commerce_status'] == 'Active' else 0, axis=1)
        resultado_tab['Valor_Total'] = resultado_tab['Cobro Total']
    
        resultado_tab['Fecha-Mes'] = mes
    
        return resultado_tab
    
    def enviar_correo(self, archivo_excel, destinatario, remitente, contrasena):
        # Crear el mensaje
        msg = EmailMessage()
        msg.set_content(f"Hola, espero se encuentren muy bien,\n\nAdjunto el archivo donde se encuentra los resultados de las comisiones calculadas para cada empresa.\n\nSaludos")
        msg['Subject'] = 'Resultados de Comisiones'
        msg['From'] = remitente
        msg['To'] = destinatario
    
        # Adjuntar el archivo Excel
        with open(archivo_excel, 'rb') as f:
            file_data = f.read()
            file_name = f.name
    
        msg.add_attachment(file_data, maintype='application', subtype='octet-stream', filename=file_name)
    
        # Conectar al servidor SMTP de Outlook y enviar el mensaje
        smtp_server = 'smtp.office365.com'
        smtp_port = 587
    
        try:
            with smtplib.SMTP(smtp_server, smtp_port) as server:
                server.starttls()  # Establecer una conexiÃ³n segura
                server.login(remitente, contrasena)
                server.send_message(msg)
                print(f"Correo electrónico enviado exitosamente a {destinatario}.")
        except Exception as e:
            print(f"Error al enviar el correo a {destinatario}: {e}")
            
            

Con la definición de las funciones dentro de la clase Calculo_comisiones, procedemos a su ejecución. La ventaja clave de utilizar una clase parametrizada es la capacidad de ajustar dinámicamente los parámetros según las necesidades del negocio. Esto significa que, ante cambios en valores críticos como la fecha de interés, el costo por petición exitosa, cambios de descuentos, umbrales de cobro o el IVA, el sistema permitirá realizar ajustes inmediatos sin la necesidad de modificar el código o recalcular manualmente los campos de interés, solo será necesario cambiar la información en los diccionarios proporcionados.

In [42]:
# Conectar a la base de datos SQLite

conn = sqlite3.connect('C:/Prueba_ecosistemas/database.sqlite')
calculo = Calculo_comisiones(conn)

In [43]:
precios = {
    'Innovexa Solutions': 300,
    'NexaTech Industries': [250,  # Asignar si hay entre 0 a 10.000 peticiones totales
                            200,  # Asignar si hay entre 10.001 a 20.000 peticiones totales
                            170], # Asignar si hay más de 20.001 peticiones totales
    'QuantumLeap Inc.': 600, 
    'Zenith Corp.': [250,  # Asignar si hay entre 0 a 22.000 peticiones totales
                     130], # Asignar si hay más de 22.001 peticiones totales
    'FusionWave Enterprises': 300
}
limites = {
    'NexaTech Industries': [10000, 20000],  # Límites de peticiones totales
    'Zenith Corp.': 22000  # Límites de peticiones totales
}
descuentos = {
    'FusionWave Enterprises': [2500, 4500],  # Rango para descuento
    'Zenith Corp.': 6000  # Valor para descuento
}
porcentajes = {
    'FusionWave Enterprises': [0.05, 0.08],  # Rango para descuento
    'Zenith Corp.': 0.05  # Valor para descuento
}


iva = 0.19 # Se añade el valor del IVA colombiano
mes = '2024-08' # Se añade el mes que se quiere desglosar, como ejemplo tomamos Julio del 2024

In [44]:
# Calcular comisiones segpun la información suministrada anteriormente

resultado_tab = calculo.calcular_comision(mes, precios, iva)

In [45]:
archivo_excel = 'resultados_comisiones.xlsx'
df_final = resultado_tab[['Fecha-Mes', 'commerce_name', 'commerce_nit', 'Valor_comision', 'Valor_iva', 'Valor_Total', 'commerce_email']]
df_final = df_final.rename(columns={'commerce_name': 'Nombre', 'commerce_nit': 'Nit', 'commerce_email': 'Correo'})
df_final.to_excel(archivo_excel, index=False)

In [46]:
df_final

,Fecha-Mes,Nombre,Nit,Valor_comision,Valor_iva,Valor_Total,Correo
0,2024-08,Zenith Corp.,28960112,29712988.5,5.645468e+06,3.535846e+07,zenithcorp.@gemaily.net
1,2024-08,FusionWave Enterprises,919341007,66103932.0,1.255975e+07,7.866368e+07,fusionwaveenterprises@microfitsof.com
2,2024-08,Innovexa Solutions,445470636,0.0,0.000000e+00,0.000000e+00,innovexasolutions@microfitsof.com
3,2024-08,QuantumLeap Inc.,198818316,143985600.0,2.735726e+07,1.713429e+08,quantumleapinc.@gemaily.net
4,2024-08,NexaTech Industries,452680670,40865450.0,7.764436e+06,4.862989e+07,nexatechindustries@gemaily.net


In [52]:
#Ahora enviamos un correo para el remitente con sus respectivos calculos de comisiones

load_dotenv(path_env)
correo_remitente = os.getenv('CORREO_REMITENTE')
correo_ejecutor = os.getenv('CORREO_EJECUTOR')
password = os.getenv('PASSWORD')

In [53]:
date_str = str(mes)
date_str

# Agrega la fecha al nombre de los archivos de salida
xlsx_filename = os.path.join(path_resultados, f'Cobro_comisión_{date_str}.xlsx')
df_final.to_excel(xlsx_filename, index=False, header=False)

In [54]:
# Enviar el correo como ejecutor

# Guardar el DataFrame como archivo Excel general
archivo_excel = xlsx_filename

# Enviar el correo como ejecutor
calculo.enviar_correo(archivo_excel, correo_ejecutor, correo_remitente, password)

Correo electrónico enviado exitosamente a danielvallejo20@hotmail.com.


Esta automatización ofrece una serie de ventajas estratégicas para el negocio. Por ejemplo, reduce significativamente la necesidad de realizar cálculos manuales, liberando tiempo para que el personal y los recursos pertinentes puedan enfocarse en otras tareas. Además, se mitiga el riesgo de errores humanos, ya que los resultados son consistentes y estandarizados, eliminando la variabilidad que puede surgir en los cálculos manuales, donde es fácil equivocarse tanto en la parte conceptual como en la práctica. El modelo parametrizado está diseñado para adaptarse fácilmente a cambios futuros, lo que facilita la actualización de condiciones comerciales sin afectar la estructura del código. Con estos cambios, se espera un impacto significativamente positivo en la eficiencia operativa, mejorando tanto los indicadores económicos de la empresa como la apertura a nuevos clientes y la fidelización de los ya existentes.

Evolución futura de la propuesta: Para mejorar la implementación en proyectos futuros, se podría considerar las siguientes aspectos:

- Parametrizar los campos de los contratos variables desde un archivo JSON para optimizar su ejecución.
- Proponer una función que permita ejecutar por medio de consola el diligenciamiento del correo del usuario y su contraseña a la hora de ejecutar el código python.


Modificaciones e implementaciones a futuro:

Este proceso puede ser implementado en un entorno productivo a futuro, ya sea mediante una calendarización interna en la empresa o migrándolo a un servidor en la nube. Esto permitiría interactuar con una variedad de herramientas digitales que ofrecen servicios complementarios, ampliando el impacto estratégico. Por ejemplo, se podrían crear repositorios de almacenamiento y generar reportes visuales en Power BI, detallando de manera clara la información entrante como los resultados para generar insides de valor comercial. Además, se podría analizar la implementación de chatbots, lo que permitiría a BATSEJ OPEN FINANCE S.A proporcionar información valiosa en tiempo real a sus aliados estratégicos, mejorando la experiencia de usuario y optimizando las consultas.